In [16]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score , precision_score
import os, sys, gc, warnings, random, datetime, math

In [17]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [18]:
SEED = 42
seed_everything(SEED)

In [19]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')
sub = pd.read_csv('PJT002_submission.csv')
gyeong =pd.concat([train, val])
data = pd.concat([train, val, test])

In [20]:
data = data.reset_index()
data.drop('index',axis= 1, inplace = True)

In [21]:
gyeong = gyeong.reset_index()
gyeong.drop('index',axis= 1, inplace = True)

In [22]:
TARGET = 'fr_yn'
LOCAL_TEST = False

In [23]:
lgb_params = {
                    'boosting_type':'gbdt',
                    'n_jobs':-1,
                    'learning_rate':0.01,
                    'num_leaves': 250,
                    'max_depth':8,
                    'n_estimators':10000,
                    'seed' :SEED,
                    'early_stopping_rounds':100
                }

In [24]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, recall_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat)
    return 'f1_score', f1_score(y_true, y_hat), True

In [25]:
rm_cols = ['id']

In [26]:
features_columns = [col for col in list(data) if col not in rm_cols]

In [27]:
train = data[:(len(train)+len(val))]
test = data[-len(test):]
dataset = [train, test]

for df in dataset:
    categorical_cols1 = df.select_dtypes(['object']).columns
    for col in categorical_cols1:
        df[col] = pd.Categorical(df[col]).codes

In [28]:
def make_predictions(tr_df, tt_df, features_columns, target, lgb_params, NFOLDS=2):
    folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  

    tt_df = tt_df[[target]]    
    predictions = np.zeros(len(tt_df))
    oof = np.zeros(len(tr_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
        tr_data = lgb.Dataset(tr_x, label=tr_y)

        if LOCAL_TEST:
            vl_data = lgb.Dataset(P, label=P_y) 
        else:
            vl_data = lgb.Dataset(vl_x, label=vl_y)  

        estimator = lgb.train(
            lgb_params,
            tr_data,
            feval=lgb_f1_score,
            valid_sets = [tr_data, vl_data],
            verbose_eval = 200,
        )   
        
        pp_p = estimator.predict(P)
        predictions += pp_p/NFOLDS
        oof_preds = estim.feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
        gc.collect()
        
    tt_df['prediction'] = predictions
    print('OOF f1score:', f1_score(y, oof))
    
    return tt_df

In [34]:
train['fr_yn']

0        1
1        0
2        1
3        0
4        0
5        1
6        0
7        0
8        0
9        0
10       1
11       0
12       1
13       0
14       1
15       0
16       1
17       0
18       1
19       0
20       1
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
66067    0
66068    0
66069    0
66070    0
66071    0
66072    0
66073    0
66074    0
66075    0
66076    0
66077    0
66078    0
66079    0
66080    0
66081    0
66082    0
66083    0
66084    1
66085    0
66086    0
66087    0
66088    0
66089    0
66090    0
66091    0
66092    0
66093    0
66094    1
66095    0
66096    1
Name: fr_yn, Length: 66097, dtype: int8

In [13]:
make_predictions (train,test, features_columns, TARGET, lgb_params,NFOLDS=2)

Fold: 0
33048 33049


LightGBMError: Length of label is not same with #data